# 2 - Data Wrangling

**Author:** M. Görkem Ulutürk

**Date:** December, 2025

## Introduction

In the previous section of the project, we conducted initial data
wrangling. We've discovered that the data contains no duplicates, but we have
missing values that we need to deal with.

Let's start by importing the modules and the data.

## Imports

In [1]:
import pandas as pd

df = pd.read_csv("../data/raw/train.csv", encoding="utf-8")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Data Cleaning & Validation

Recall that we've already checked for duplicates in the understanding and
planning phase, and we've found no duplicates. Let's start by converting
the column names to lowercase.

In [2]:
df.columns = df.columns.str.lower()

Let's also validate data types.

In [3]:
df.dtypes

passengerid      int64
survived         int64
pclass           int64
name            object
sex             object
age            float64
sibsp            int64
parch            int64
ticket          object
fare           float64
cabin           object
embarked        object
dtype: object

Data types are correct. Then, we can check for invalid values. For example,
we expect `survived` to be only 0 or 1.

In [4]:
df["sex"].value_counts(dropna=False)

sex
male      577
female    314
Name: count, dtype: int64

In [5]:
df["embarked"].value_counts(dropna=False)

embarked
S      644
C      168
Q       77
NaN      2
Name: count, dtype: int64

In [6]:
df[["survived", "pclass", "age", "sibsp", "parch", "fare"]].agg(["min", "max"])

,survived,pclass,age,sibsp,parch,fare
min,0,1,0.42,0,0,0.0000
max,1,3,80.00,8,6,512.3292


We see no invalid values among these columns except for `fare`, where there
exist rows with a `fare` amount of 0. These can be real values since a
ticket could be handed for free, so we'll check these when we're dealing
with outliers. For now, all values seem to be valid.

Let's also validate the `cabin` column. During the understanding phase, we've
discovered that this column includes quite a lot of missing values. Before we
attempt to fill those values in, let's validate the existing ones.

Titanic accommodated luxurious cabins with utmost comfort, especially for
the first-class passengers. Below is a cutaway diagram depicting these
facilities.

<center>
<div>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/0d/Olympic_%26_Titanic_cutaway_diagram.png/960px-Olympic_%26_Titanic_cutaway_diagram.png" alt="Titanic cutaway diagram" height="500">
</div>
</center>

> "The accomodation for first-class passengers is placed amidships and
extends over five decks, the promenade (A), bridge (B), shelter (C), saloon
(D), and upper (E) decks. *(Titanica, 2025)*"

First-class accommodations were placed in cabins A through E, second and
third-classes in D through G.

In our dataset, we see the following cases:

In [7]:
cabins = df.loc[df["cabin"].notna(), "cabin"].unique()
cabins[:70]

array(['C85', 'C123', 'E46', 'G6', 'C103', 'D56', 'A6', 'C23 C25 C27',
       'B78', 'D33', 'B30', 'C52', 'B28', 'C83', 'F33', 'F G73', 'E31',
       'A5', 'D10 D12', 'D26', 'C110', 'B58 B60', 'E101', 'F E69', 'D47',
       'B86', 'F2', 'C2', 'E33', 'B19', 'A7', 'C49', 'F4', 'A32', 'B4',
       'B80', 'A31', 'D36', 'D15', 'C93', 'C78', 'D35', 'C87', 'B77',
       'E67', 'B94', 'C125', 'C99', 'C118', 'D7', 'A19', 'B49', 'D',
       'C22 C26', 'C106', 'C65', 'E36', 'C54', 'B57 B59 B63 B66', 'C7',
       'E34', 'C32', 'B18', 'C124', 'C91', 'E40', 'T', 'C128', 'D37',
       'B35'], dtype=object)

  - Cabin numbers start with a capital English letter
  - Some passengers accommodated multiple cabins
      - This is likely due to cabin numbers being shared between passengers
      groups such as families (similar to the ticket number)
  - Some passengers have cabin numbers starting with F, followed by another
  cabin string, eg, F G73
  - There exists a "T" cabin
  - Some passengers don't have full cabin numbers but just the deck letter

Firstly, let's investigate the cabin T.

In [8]:
df.loc[df.cabin.str.contains("T") == True, "name"]

339    Blackwell, Mr. Stephen Weart
Name: name, dtype: object

There's only 1 passenger with this cabin number, Mr. Stephen Weart Blackwell.
According to Titanica, Mr. Blackwell was the only passenger with the cabin
number T[1], and this number referred to the boat deck[2]. Thus, this is a
legitimate entry.

Now, the entries like "F G73" are not mistakes either. The reason is that
this cabin number denotes the deck F, section G, cabin 73. We infer this
information from the Titanic's plans themselves[3].

We're not yet sure whether cabin numbers themselves are correlated with the
survival of a passenger, though the deck information of the cabin number
could be. Therefore, we'll create a new column called `deck` that we'll fill
in by extracting it from the cabin number itself. This approach has 3 main
benefits:

1. We can fill in missing cabin numbers more reliably
2. Reduces model complexity by focusing on a broader feature
3. Reduces the risk of overfitting

In [9]:
def extract_deck(cabin: str) -> str | list:
    """
    Extracts the deck information (A, B, C, etc.) from the cabin number.

    Parameters
    ----------
    cabin : str
        Cabin number

    Returns
    -------
    str
        Cabin's deck
    """
    lst = cabin.split(" ")
    if len(lst) == 1:  # cabin number is of type letter + number
        return lst[0][0]
    if all([lst[0][0] == x[0] for x in lst]):
        return lst[0][0]
    if lst[0][0] == "F" and lst[1][0] in [
        "G",
        "E",
    ]:
        return f"F-{lst[1][0]}"
    return lst

In [10]:
df[df["cabin"].notna()]["cabin"].map(extract_deck).unique()

array(['C', 'E', 'G', 'D', 'A', 'B', 'F', 'F-G', 'F-E', 'T'], dtype=object)

We see that only F G000 and F E000 types of cabin numbers exist, while there
are other such possibilities, such as F R000. Although some sections, such as
F-J did not contain any rooms; some sections that had rooms are still not
present in the dataset. Therefore, instead of distinguishing between a deck's
sections, we'll just use the main deck string and exclude the section.

In [11]:
def extract_deck_sectionless(cabin: str) -> str | list:
    """
    Extracts the deck information (A, B, C, etc.) from the cabin number.
    Ignores the sections such as F G73, F R171, etc. and only returns the deck
    letter.

    Parameters
    ----------
    cabin : str
        Cabin number

    Returns
    -------
    str
        Cabin's deck
    """
    lst = cabin.split(" ")
    if len(lst) == 1:
        return lst[0][0]
    if all([lst[0][0] == x[0] for x in lst]):
        return lst[0][0]
    if lst[0][0] == "F" and lst[1][0] in [
        "G",
        "E",
    ]:
        return f"F"  # Only this line is modified
    return lst


df["deck"] = df.loc[df["cabin"].notna(), "cabin"].map(extract_deck_sectionless)
df.loc[df["cabin"].notna(), ["cabin", "deck"]].head(10)

,cabin,deck
1,C85,C
3,C123,C
6,E46,E
10,G6,G
11,C103,C
21,D56,D
23,A6,A
27,C23 C25 C27,C
31,B78,B
52,D33,D


Notice that our assumption that the group tickets' cabin numbers were all in
the same deck was true for the non-missing data. We'll keep this assumption
in mind when filling in the missing values for the cabin column.

Lastly, let's check if the assumption that the first-class accommodations are
in decks A through E, and second and third classes in decks D through G.

In [12]:
invalid = (
    "(pclass == 1 and deck in ['F', 'G']) or "
    "(pclass == 2 and deck in ['A', 'B', 'C']) or "
    "(pclass == 3 and deck in ['A', 'B', 'C'])"
)

mismatched = df.dropna(subset=["cabin"]).query(invalid)
mismatched[["passengerid", "pclass", "deck", "cabin"]]

,passengerid,pclass,deck,cabin


There are no mismatches between passenger classes and their decks. Thus, the
`cabin` column contains no invalid entries.

Lastly, we can validate the `sex` column using passenger names. Firstly, we
can extract passenger titles from the `name` column. To do that, we need to
extract what titles are present in the dataset. What makes it easy in our
case is that titles end with a dot at the end of the word, so we can filter
by using that.

In [13]:
titles = []
for index, row in df.iterrows():
    for _str in row["name"].split(" "):
        if "." in _str and _str not in titles:
            titles.append(_str)
titles

['Mr.',
 'Mrs.',
 'Miss.',
 'Master.',
 'Don.',
 'Rev.',
 'Dr.',
 'Mme.',
 'Ms.',
 'Major.',
 'L.',
 'Lady.',
 'Sir.',
 'Mlle.',
 'Col.',
 'Capt.',
 'Countess.',
 'Jonkheer.']

Apart from one mistake in the titles ("L."), the rest of them are valid
titles in English, French, Italian, etc. The "L." comes from a shortened name
in the dataset. Below is an SQL query for the string "L."

![SQL Name column querry](../assets/sql_name_querry.png)

These titles can be explained as follows:
- "Mr.": for men, regardless of marital status, who do not have another
professional or academic title
- "Mrs.": for married women who do not have another professional or academic
title
- "Miss.": for girls, unmarried women, and (in the United Kingdom) married
women who continue to use their maiden name
- "Master.": for boys and young men, or as a style for the heir to a Scottish
peerage
- "Don.": commonly used in Spain, Portugal, and Italy, it is an honorific
prefix derived from the Latin Dominus, meaning "lord" or "owner"
- "Rev.": used generally for members of the Christian clergy, regardless of
affiliation, but especially in Catholic and Protestant denominations, for
both men and women
- "Dr.": for the holder of a doctoral degree in many countries, and for
medical practitioners, dentists, and veterinary surgeons
- "Mme.": the French abbreviation for Madame, for women, a term of general
respect or flattery, originally used only for a woman of rank or authority
- "Ms.": for women, regardless of marital status or when marital status is
unknown
- "Major.": a military title
- "Lady.": for female peers with the rank of baroness, viscountess, countess,
and marchioness, or the wives of men who hold the equivalent titles
- "Sir.": for men, formally, if they have a British knighthood or if they are
a baronet
- "Mlle.": is a French courtesy title traditionally given to an unmarried
woman
- "Col.": a military title
- "Capt.": a military title or a ship's highest responsible officer acting on
behalf of the ship's owner
- "Countess.": a woman of high social rank, or the wife of a count or earl
- "Jonkheer.": is literally translated as 'young lord' in Dutch

(Paraphrased or directly quoted from the references [4], [5], [6], [7], [8],
[9])

Let's now create a column `title` from the names.

In [14]:
titles = [
    "Mr.",
    "Mrs.",
    "Miss.",
    "Master.",
    "Don.",
    "Rev.",
    "Dr.",
    "Mme.",
    "Ms.",
    "Major.",
    "Lady.",
    "Sir.",
    "Mlle.",
    "Col.",
    "Capt.",
    "Countess.",
    "Jonkheer.",
]


def extract_title(name: str) -> str:
    """
    Extracts the honorific title from the name string

    Parameters
    ----------
    name : str
        Passenger name

    Returns
    -------
    str
        Honorific title
    """
    str_list = name.split(" ")
    for _str in str_list:
        if _str in titles:
            return _str
    return ""

In [15]:
df["title"] = df["name"].apply(extract_title)

df[["name", "title"]].head(10)

,name,title
0,"Braund, Mr. Owen Harris",Mr.
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",Mrs.
2,"Heikkinen, Miss. Laina",Miss.
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",Mrs.
4,"Allen, Mr. William Henry",Mr.
5,"Moran, Mr. James",Mr.
6,"McCarthy, Mr. Timothy J",Mr.
7,"Palsson, Master. Gosta Leonard",Master.
8,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",Mrs.
9,"Nasser, Mrs. Nicholas (Adele Achem)",Mrs.


Now, we can cross-reference titles with their respective genders. Note that
some of the titles in our dataset are gender-neutral. Thus, we'll only check
for the ones that are gender-specific initially.

In [16]:
pd.crosstab(df["sex"], df["title"])

title,Capt.,Col.,Countess.,Don.,Dr.,Jonkheer.,Lady.,Major.,Master.,Miss.,Mlle.,Mme.,Mr.,Mrs.,Ms.,Rev.,Sir.
sex,,,,,,,,,,,,,,,,,
female,0,0,1,0,1,0,1,0,0,182,2,1,0,125,1,0,0
male,1,2,0,1,6,1,0,2,40,0,0,0,517,0,0,6,1


We see that the gender-specific titles all match with a passenger's sex. On
the other hand, we cannot methodically check for gender-neutral honorifics
and that the only way to verify those would be to individually research the
passengers with these titles. This verification is out of the scope of this
project and will not be needed for our purposes. Therefore, we deem that
we've verified the `sex` column using the passenger titles we've extracted
from their names.